In [4]:
import pandas as pd
import numpy as np
import yfinance as yf

# Fetch market data
def get_data(ticker):
    data = yf.download(ticker, start="2020-01-01", end="2024-01-01")
    return data

# Preprocess the data
def preprocess_data(data):
    data = data.dropna()
    data['Return'] = data['Close'].pct_change()
    data['Direction'] = np.where(data['Return'] > 0, 1, 0)
    return data

data = get_data('AAPL')
processed_data = preprocess_data(data)


[*********************100%***********************]  1 of 1 completed


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Feature engineering
def feature_engineering(data):
    data['MA10'] = data['Close'].rolling(window=10).mean()
    data['MA50'] = data['Close'].rolling(window=50).mean()
    data = data.dropna()
    return data

# Model training
def train_model(data):
    X = data[['MA10', 'MA50', 'Volume']]
    y = data['Direction']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = RandomForestClassifier(n_estimators=100)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return model, accuracy

data = feature_engineering(processed_data)
model, accuracy = train_model(data)
print(f"Model Accuracy: {accuracy:.2f}")


Model Accuracy: 0.57


In [ ]:
// Solidity version
pragma solidity ^0.8.0;

contract MarketPrediction {
    address public owner;
    string public prediction;

    event NewPrediction(string _prediction);

    constructor() {
        owner = msg.sender;
    }

    function setPrediction(string memory _prediction) public {
        require(msg.sender == owner, "Only owner can set prediction");
        prediction = _prediction;
        emit NewPrediction(_prediction);
    }

    function getPrediction() public view returns (string memory) {
        return prediction;
    }
}


In [ ]:
import Web3 from 'web3';

const web3 = new Web3(Web3.givenProvider || "http://localhost:8545");
const contractABI = [...];  // Replace with actual ABI
const contractAddress = "0x...";  // Replace with actual contract address

const contract = new web3.eth.Contract(contractABI, contractAddress);

async function setPrediction(prediction) {
    const accounts = await web3.eth.getAccounts();
    await contract.methods.setPrediction(prediction).send({ from: accounts[0] });
}

async function getPrediction() {
    const prediction = await contract.methods.getPrediction().call();
    return prediction;
}


In [ ]:
import React, { useState, useEffect } from 'react';
import { getPrediction, setPrediction } from './web3';

function App() {
    const [prediction, setPredictionState] = useState('');

    useEffect(() => {
        const fetchPrediction = async () => {
            const pred = await getPrediction();
            setPredictionState(pred);
        };
        fetchPrediction();
    }, []);

    const handleSetPrediction = async () => {
        await setPrediction(prediction);
    };

    return (
        <div>
            <h1>Decentralized Market Prediction</h1>
            <div>
                <h2>Current Prediction: {prediction}</h2>
                <input type="text" onChange={(e) => setPredictionState(e.target.value)} />
                <button onClick={handleSetPrediction}>Set Prediction</button>
            </div>
        </div>
    );
}

export default App;


In [ ]:
from flask import Flask, request, jsonify
import joblib

app = Flask(__name__)

model = joblib.load('market_model.pkl')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    prediction = model.predict([data['features']])
    return jsonify({'prediction': int(prediction[0])})

if __name__ == '__main__':
    app.run(debug=True)
